In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import geopandas as gpd
from geopandas import GeoDataFrame
import xlrd
from random import randint
import requests
from io import BytesIO
from datetime import datetime

pd.set_option('display.max_columns', None)

## Reading spatial and excel files for 2025:

In [2]:
gdf = gpd.read_file("./shp/SF_DOE_PREC_2022_07_18_pg.shp", encoding='utf-8')
gdf = gdf.to_crs('EPSG:4269')

In [3]:
url = 'https://www.sfelections.org/results/20250916/data/20250807/sov.xlsx'

In [4]:
# date = datetime.today().strftime("%Y%m%d")

# election_date = "20250916"

# # Build the URL
# url = f"https://www.sfelections.org/results/{election_date}/data/{date}/sov.xlsx"

print(url)

https://www.sfelections.org/results/20250916/data/20250807/sov.xlsx


In [5]:
response = requests.get(url)

In [6]:
dfTurnout = pd.read_excel(BytesIO(response.content), sheet_name='Sheet1', header=4, skipfooter=10)
dfA = pd.read_excel(BytesIO(response.content), sheet_name='Sheet2', header=3, skipfooter=8)

In [7]:
dfA

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,Yes\n,Unnamed: 7,No\n,Unnamed: 9,Total Votes,Unnamed: 11
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
4,Vote by Mail,2566.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
80,Vote by Mail,2383.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN


### Formatting

In [8]:
###
### TURNOUT
###

#trim and rename columns
dfTurnout = dfTurnout[['Precinct','Registered\nVoters','Voters Cast']]
dfTurnout.columns = ['precinct','registered_voters','votes_cast']

# shift precinct column

dfTurnout['precinct'] = dfTurnout['precinct'].shift(3)

#drop unnecessary ones
dfTurnout = dfTurnout.dropna()
dfTurnout = dfTurnout[dfTurnout["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfTurnout['precinct'] = dfTurnout['precinct'].str.replace('PCT ','').str.replace(' MB','')

#change data types
dfTurnout.registered_voters = dfTurnout.registered_voters.astype(int)
dfTurnout.votes_cast = dfTurnout.votes_cast.astype(int)

#create turnout column
dfTurnout['turnout'] = round((dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100, 1)
dfTurnout['turnout'] = dfTurnout['turnout'].fillna(0)

In [9]:
dfTurnout

,precinct,registered_voters,votes_cast,turnout
5,9401,2566,0,0.0
9,9403,2247,0,0.0
13,9404,3243,0,0.0
17,9406,2415,0,0.0
21,9408,2979,0,0.0
25,9413,3296,0,0.0
29,9414,2227,0,0.0
33,9417,2211,0,0.0
37,9421,2106,0,0.0
41,9423,2363,0,0.0


In [10]:
###
### PROPS
###

def process_proposition(df):
    # Standardize column names
    df.columns = df.columns.str.replace('\n', '', regex=True)
    df.columns = df.columns.str.replace('"', '', regex=True)
    df.columns = df.columns.str.replace('.', '', regex=True)
    df.columns = df.columns.str.lower().str.replace(' ', '_', regex=True)
    df.columns = df.columns.str.replace('_$', '', regex=True)

    # if the column name contains "bonds_-_yes", replace it with "yes"

    df.columns = df.columns.str.replace('bonds_-_yes', 'yes', regex=True)
    df.columns = df.columns.str.replace('bonds_-_no', 'no', regex=True)
    
    # Trim and rename columns to focus on necessary data
    df = df[['precinct', 'registered_voters', 'yes', 'no', 'total_votes']]
    df.columns = ['precinct', 'registered_voters', 'yes', 'no', 'votes_cast']

    # shift the precinct column up by 3 rows

    df['precinct'] = df['precinct'].shift(3)

    # Drop rows where precinct is NaN after the shift
    df = df.dropna(subset=['precinct'])
    df = df[df['precinct'].str.contains('PCT', na=False)]

    # Clean up precinct column data
    df['precinct'] = df['precinct'].str.replace('PCT ', '', regex=True).str.replace(' MB', '', regex=True)

    # Ensure numeric columns are of the correct type
    # df['registered_voters'] = pd.to_numeric(df['registered_voters'], errors='coerce').fillna(0).astype(int)
    # df['votes_cast'] = pd.to_numeric(df['votes_cast'], errors='coerce').fillna(0).astype(int)

    # Calculate turnout
    df['turnout'] = round((df['votes_cast'] / df['registered_voters']) * 100, 1).fillna(0)

    return df

In [11]:
dfA = process_proposition(dfA)

KeyError: "None of [Index(['precinct', 'registered_voters', 'yes', 'no', 'total_votes'], dtype='object')] are in the [columns]"

In [12]:
bins = [0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 100]
labels = ['Less than 25%', '25-30%', '30-35%', '35-40%', '40-45%', '45-50%', '50-55%', '55-60%', '60-65%', '65-70%', '70-75%', '75% and more']

In [13]:
# I want to generate random numbers for yes and no columns in dfA, but the sum of yes and no should be  no more than registered_voters

def generate_yes_no(row):
    registered_voters = row['registered_voters']
    if registered_voters == 0:
        return pd.Series([0, 0])
    
    yes = randint(0, registered_voters)
    no = randint(0, registered_voters - yes)
    
    return pd.Series([yes, no])

# make votes_cast the sum of yes and no

dfA[['yes', 'no']] = dfA.apply(generate_yes_no, axis=1)
dfA['votes_cast'] = dfA['yes'] + dfA['no']
dfA['turnout'] = round((dfA['votes_cast'] / dfA['registered_voters']) * 100, 1).fillna(0)

KeyError: 'registered_voters'

In [14]:
dfTurnout.head()

,precinct,registered_voters,votes_cast,turnout
5,9401,2566,0,0.0
9,9403,2247,0,0.0
13,9404,3243,0,0.0
17,9406,2415,0,0.0
21,9408,2979,0,0.0


In [15]:
# do the same thing for dfTurnout. This time we are just populating the votes_cast column with random numbers

dfTurnout['votes_cast'] = dfTurnout.apply(
    lambda row: randint(0, row['registered_voters']) if row['registered_voters'] > 0 else 0,
    axis=1
)
dfTurnout['turnout'] = round((dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100, 1).fillna(0)

In [16]:
dfA

,,,,,,,,,,,,
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
4,Vote by Mail,2566.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
80,Vote by Mail,2383.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN


In [17]:

# Calculate voter turnout percentage
dfTurnout['yes_perc'] = (dfTurnout['votes_cast'] / dfTurnout['registered_voters']) * 100

# Categorize turnout percentages into bins
dfTurnout['yes_perc'] = pd.cut(dfTurnout['yes_perc'], bins=bins, labels=labels, include_lowest=True)

In [18]:
dfA['precinct'] = dfA['precinct'].astype(int)
dfA['yes_perc'] = (dfA['yes'] / dfA['votes_cast']) * 100
dfA['yes_perc_bin'] = pd.cut(dfA['yes_perc'], bins=bins, labels=labels, include_lowest=True)
dfA['yes_perc'] = dfA['yes_perc'].fillna('no data')
dfA['yes_perc_bin'] = dfA['yes_perc_bin'].astype(str)

KeyError: 'precinct'

In [19]:
dfTurnout

,precinct,registered_voters,votes_cast,turnout,yes_perc
5,9401,2566,1505,58.7,55-60%
9,9403,2247,240,10.7,Less than 25%
13,9404,3243,1704,52.5,50-55%
17,9406,2415,1003,41.5,40-45%
21,9408,2979,2196,73.7,70-75%
25,9413,3296,55,1.7,Less than 25%
29,9414,2227,2041,91.6,75% and more
33,9417,2211,930,42.1,40-45%
37,9421,2106,1517,72.0,70-75%
41,9423,2363,1586,67.1,65-70%


## Reading files for 2022

In [20]:
url = 'https://www.sfelections.org/results/20221108/data/20221201/sov.xlsx'

In [21]:
response = requests.get(url)

In [22]:
dfd4 = pd.read_excel(BytesIO(response.content), sheet_name='Sheet43', header=3, skipfooter=8)

In [23]:
dfd4.head()

,Precinct,Registered \nVoters,Undervotes,Unnamed: 3,Overvotes,Precinct.1,JOEL ENGARDIO\n,Unnamed: 7,GORDON MAR\n,Unnamed: 9,Write-in\n,Unnamed: 11,Unnamed: 12,Total Votes
0,Countywide,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,1215.0,9.0,NaN,0.0,Election Day,44.0,41.90%,61.0,58.10%,0.0,NaN,0.00%,105.0
4,Vote by Mail,1215.0,56.0,NaN,0.0,Vote by Mail,223.0,38.65%,354.0,61.35%,0.0,NaN,0.00%,577.0


In [24]:
### DISTRICT 4

# make all columns lowercase and remove spaces
dfd4.columns = dfd4.columns.str.replace('\n', '')
dfd4.columns = dfd4.columns.str.lower().str.replace(' ', '_')
dfd4.columns = dfd4.columns.str.replace('_$', '')

dfd4['precinct'] = dfd4['precinct'].shift(3)

dfd4 = dfd4[['precinct', 'registered_voters', 'joel_engardio', 'gordon_mar','total_votes']]

#drop unnecessary ones
dfd4 = dfd4.dropna()
dfd4 = dfd4[dfd4["precinct"].str.contains('PCT')]

#tidy up precinct column data
dfd4['precinct'] = dfd4['precinct'].str.replace('PCT ','').str.replace(' MB','')

KeyError: "['joel_engardio', 'gordon_mar'] not in index"

In [25]:
groupings = {
    "9401": ["9401", "9402"], 
    "9403": ["9403", "9411"],
    "9404": ["9404", "9405", "9412"],
    "9406": ["9406", "9407"],
    "9408": ["9408", "9409", "9416"],
    "9413": ["9413", "9418", "9419"], 
    "9414": ["9414", "9415"],
    "9417": ["9417", "9424"],
    "9421": ["9421", "9422"], 
    "9423": ["9423", "9428"], 
    "9425": ["9425", "9426", "9431"],
    "9427": ["9427", "9433", "9434"],
    "9429": ["9429", "9436", "9438"], 
    "9432": ["9432", "9441"], 
    "9435": ["9435", "9443"],
    "9437": ["9437", "9444", "9445"],
    "9439": ["9439","9446"], 
    "9442": ["9442", "9447", "9448"],
    "9449": ["9449"],
    "9451": ["9451", "9452"]
}

In [26]:
dfd4

,precinct,registered_voters,undervotes,unnamed:_3,overvotes,precinct.1,joel_engardio_,unnamed:_7,gordon_mar_,unnamed:_9,write-in_,unnamed:_11,unnamed:_12,total_votes
0,None,NaN,NaN,NaN,NaN,Countywide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,None,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,None,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Countywide,1215.0,9.0,NaN,0.0,Election Day,44.0,41.90%,61.0,58.10%,0.0,NaN,0.00%,105.0
4,Electionwide,1215.0,56.0,NaN,0.0,Vote by Mail,223.0,38.65%,354.0,61.35%,0.0,NaN,0.00%,577.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,PCT 9451,1168.0,96.0,NaN,1.0,Total,489.0,69.66%,213.0,30.34%,0.0,NaN,0.00%,702.0
186,Election Day,NaN,NaN,NaN,NaN,PCT 9452,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
187,Vote by Mail,1052.0,8.0,NaN,0.0,Election Day,32.0,72.73%,12.0,27.27%,0.0,NaN,0.00%,44.0
188,Total,1052.0,58.0,NaN,0.0,Vote by Mail,361.0,66.61%,181.0,33.39%,0.0,NaN,0.00%,542.0


In [27]:
# If a precint matches a key in the groupings dictionary, add up the rows that match the values in the list and create a new row with the key as the precinct

def group_precincts(df, groupings):
    grouped_data = []

    for key, values in groupings.items():
        subset = df[df['precinct'].isin(values)]
        if not subset.empty:
            aggregated_row = {
                'precinct': key,
                'registered_voters': subset['registered_voters'].sum(),
                'joel_engardio': subset['joel_engardio'].sum(),
                'gordon_mar': subset['gordon_mar'].sum(),
                'total_votes': subset['total_votes'].sum()
            }
            grouped_data.append(aggregated_row)

    grouped_df = pd.DataFrame(grouped_data)
    return grouped_df

dfd4_grouped = group_precincts(dfd4, groupings)

In [28]:
dfd4_grouped

""


In [29]:
dfd4_grouped["joel_engardio_p"] = round((dfd4_grouped["joel_engardio"] / dfd4_grouped["total_votes"]) * 100, 1)
dfd4_grouped["gordon_mar_p"] = round((dfd4_grouped["gordon_mar"] / dfd4_grouped["total_votes"]) * 100, 1)

KeyError: 'joel_engardio'

In [30]:
dfd4_grouped

""


In [31]:
dfd4_grouped['winner'] = dfd4_grouped[['joel_engardio_p', 'gordon_mar_p']].idxmax(axis=1).str.replace('_p', '')

KeyError: "None of [Index(['joel_engardio_p', 'gordon_mar_p'], dtype='object')] are in the [columns]"

## Prepping gdf

In [32]:
gdf = gdf[['Prec_2022','geometry']]
gdf.Prec_2022 = gdf.Prec_2022.astype(str)

In [33]:
gdf

,Prec_2022,geometry
0,7042,"POLYGON ((-122.42165 37.71029, -122.421 37.710..."
1,1107,"POLYGON ((-122.45595 37.71134, -122.45593 37.7..."
2,1145,"POLYGON ((-122.44617 37.71104, -122.4455 37.70..."
3,7043,"POLYGON ((-122.41508 37.71166, -122.41289 37.7..."
4,7046,"POLYGON ((-122.40973 37.71195, -122.40883 37.7..."
...,...,...
509,9201,"POLYGON ((-122.47754 37.81011, -122.4775 37.80..."
510,9202,"POLYGON ((-122.47725 37.81102, -122.47711 37.8..."
511,7301,"POLYGON ((-122.41991 37.81163, -122.4199 37.81..."
512,7645,"POLYGON ((-122.36862 37.83116, -122.36733 37.8..."


In [34]:
groupings = {
    "9401": ["9401", "9402"], 
    "9403": ["9403", "9411"],
    "9404": ["9404", "9405", "9412"],
    "9406": ["9406", "9407"],
    "9408": ["9408", "9409", "9416"],
    "9413": ["9413", "9418", "9419"], 
    "9414": ["9414", "9415"],
    "9417": ["9417", "9424"],
    "9421": ["9421", "9422"], 
    "9423": ["9423", "9428"], 
    "9425": ["9425", "9426", "9431"],
    "9427": ["9427", "9433", "9434"],
    "9429": ["9429", "9436", "9438"], 
    "9432": ["9432", "9441"], 
    "9435": ["9435", "9443"],
    "9437": ["9437", "9444", "9445"],
    "9439": ["9439","9446"], 
    "9442": ["9442", "9447", "9448"],
    "9449": ["9449"],
    "9451": ["9451", "9452"]
}

In [35]:
# I want to merge the precincts in gdf based on the groupings above. The new precinct column should be the key of the groupings dict

gdf['precinct'] = gdf['Prec_2022'].astype(str)

gdf['precinct'] = gdf['precinct'].apply(lambda x: next((key for key, values in groupings.items() if x in values), x))

gdf = gdf.dissolve(by='precinct', as_index=False).reset_index(drop=True)

gdf

,precinct,geometry,Prec_2022
0,1101,"POLYGON ((-122.47167 37.72163, -122.47078 37.7...",1101
1,1102,"POLYGON ((-122.46802 37.7161, -122.468 37.7142...",1102
2,1103,"POLYGON ((-122.46625 37.71968, -122.46715 37.7...",1103
3,1104,"POLYGON ((-122.46266 37.72, -122.46267 37.7199...",1104
4,1105,"POLYGON ((-122.46263 37.71742, -122.46262 37.7...",1105
...,...,...,...
482,9808,"POLYGON ((-122.44106 37.73497, -122.44031 37.7...",9808
483,9901,"POLYGON ((-122.43251 37.73311, -122.43233 37.7...",9901
484,9902,"POLYGON ((-122.40396 37.73161, -122.4034 37.73...",9902
485,9903,"POLYGON ((-122.33081 37.78579, -122.33061 37.7...",9903


In [36]:
# export to geojson

gdf.to_file("sf_precincts_merged.geojson", driver='GeoJSON', encoding='utf-8')

# Merging

In [37]:
# make dfTurnout yes_perc a string too

dfTurnout['yes_perc'] = dfTurnout['yes_perc'].astype(str)

In [38]:
dfTurnout

,precinct,registered_voters,votes_cast,turnout,yes_perc
5,9401,2566,1505,58.7,55-60%
9,9403,2247,240,10.7,Less than 25%
13,9404,3243,1704,52.5,50-55%
17,9406,2415,1003,41.5,40-45%
21,9408,2979,2196,73.7,70-75%
25,9413,3296,55,1.7,Less than 25%
29,9414,2227,2041,91.6,75% and more
33,9417,2211,930,42.1,40-45%
37,9421,2106,1517,72.0,70-75%
41,9423,2363,1586,67.1,65-70%


In [39]:
gdf

,precinct,geometry,Prec_2022
0,1101,"POLYGON ((-122.47167 37.72163, -122.47078 37.7...",1101
1,1102,"POLYGON ((-122.46802 37.7161, -122.468 37.7142...",1102
2,1103,"POLYGON ((-122.46625 37.71968, -122.46715 37.7...",1103
3,1104,"POLYGON ((-122.46266 37.72, -122.46267 37.7199...",1104
4,1105,"POLYGON ((-122.46263 37.71742, -122.46262 37.7...",1105
...,...,...,...
482,9808,"POLYGON ((-122.44106 37.73497, -122.44031 37.7...",9808
483,9901,"POLYGON ((-122.43251 37.73311, -122.43233 37.7...",9901
484,9902,"POLYGON ((-122.40396 37.73161, -122.4034 37.73...",9902
485,9903,"POLYGON ((-122.33081 37.78579, -122.33061 37.7...",9903


In [40]:
dfTurnout = gdf.merge(dfTurnout, right_on='precinct', left_on='precinct')
dfTurnout = dfTurnout[['precinct','registered_voters','votes_cast','yes_perc','turnout','geometry']]
gdfTurnout = GeoDataFrame(dfTurnout, crs="EPSG:4269", geometry='geometry')

In [41]:
dfA.dtypes

     object
    float64
    float64
    float64
    float64
     object
    float64
    float64
    float64
    float64
    float64
    float64
dtype: object

In [42]:
# make dfA precinct string

dfA['precinct'] = dfA['precinct'].astype(str)

KeyError: 'precinct'

In [43]:
# merge 

dfA = gdf.merge(dfA, on='precinct')

KeyError: 'precinct'

In [44]:
dfA.columns

Index(['', '', '', '', '', '', '', '', '', '', '', ''], dtype='object')

In [45]:
dfA

,,,,,,,,,,,,
0,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
1,Electionwide,NaN,NaN,NaN,NaN,Electionwide,NaN,NaN,NaN,NaN,NaN,NaN
2,PCT 9401,NaN,NaN,NaN,NaN,PCT 9401,NaN,NaN,NaN,NaN,NaN,NaN
3,Election Day,2566.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
4,Vote by Mail,2566.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
77,Total,594.0,0.0,NaN,0.0,Total,0.0,NaN,0.0,NaN,0.0,NaN
78,PCT 9451,NaN,NaN,NaN,NaN,PCT 9451,NaN,NaN,NaN,NaN,NaN,NaN
79,Election Day,2383.0,0.0,NaN,0.0,Election Day,0.0,NaN,0.0,NaN,0.0,NaN
80,Vote by Mail,2383.0,0.0,NaN,0.0,Vote by Mail,0.0,NaN,0.0,NaN,0.0,NaN


In [46]:
dfA = dfA[['precinct','registered_voters','yes','no','votes_cast','turnout','yes_perc','yes_perc_bin','geometry']]
gdfA = GeoDataFrame(dfA, crs="EPSG:4269", geometry='geometry')

KeyError: "None of [Index(['precinct', 'registered_voters', 'yes', 'no', 'votes_cast', 'turnout',\n       'yes_perc', 'yes_perc_bin', 'geometry'],\n      dtype='object')] are in the [columns]"

In [47]:
# merge gdf dfd4_grouped

dfd4_grouped['precinct'] = dfd4_grouped['precinct'].astype(str)

gdf4 = gdf.merge(dfd4_grouped, on='precinct')

gdf4


KeyError: 'precinct'

In [48]:
gdf

,precinct,geometry,Prec_2022
0,1101,"POLYGON ((-122.47167 37.72163, -122.47078 37.7...",1101
1,1102,"POLYGON ((-122.46802 37.7161, -122.468 37.7142...",1102
2,1103,"POLYGON ((-122.46625 37.71968, -122.46715 37.7...",1103
3,1104,"POLYGON ((-122.46266 37.72, -122.46267 37.7199...",1104
4,1105,"POLYGON ((-122.46263 37.71742, -122.46262 37.7...",1105
...,...,...,...
482,9808,"POLYGON ((-122.44106 37.73497, -122.44031 37.7...",9808
483,9901,"POLYGON ((-122.43251 37.73311, -122.43233 37.7...",9901
484,9902,"POLYGON ((-122.40396 37.73161, -122.4034 37.73...",9902
485,9903,"POLYGON ((-122.33081 37.78579, -122.33061 37.7...",9903


In [49]:
# export both 

gdfTurnout.to_file("./docs/turnout/turnout.geojson", driver='GeoJSON', encoding='utf-8')
gdfA.to_file("./docs/propA/propA.geojson", driver='GeoJSON', encoding='utf-8')

NameError: name 'gdfA' is not defined

In [50]:
# export gdf4

gdf4.to_file("./docs/2022/d4.geojson", driver='GeoJSON', encoding='utf-8')

NameError: name 'gdf4' is not defined